<a href="https://colab.research.google.com/github/Anil-Chalumuri/TransferLearning/blob/main/Transfer_Learning_Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Transfer Learning ResNet50` using Keras

1.   List item
2.   List item



Please download the dataset from the below url

In [2]:
import tensorflow as tf 

if tf.test.gpu_device_name(): 
    

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [14]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob


In [2]:


from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [3]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'Datasets/train'
valid_path = 'Datasets/test'


In [4]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

#vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
resnet50 = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)




94773248/94765736 [==============================] - 0s 0us/step


In [5]:
# don't train existing weights
for layer in resnet50.layers:
    layer.trainable = False

In [9]:
  # useful for getting number of output classes
folders = glob('/content/drive/MyDrive/Datasets/train/*')

In [10]:
folders

['/content/drive/MyDrive/Datasets/train/cat',
 '/content/drive/MyDrive/Datasets/train/dog']

In [11]:
# our layers - you can add more if you want
x = Flatten()(resnet50.output)

In [13]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet50.input, outputs=prediction)

In [14]:

# view the structure of the model
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [15]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [64]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [65]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 25000 images belonging to 2 classes.


In [66]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1012 images belonging to 2 classes.


In [19]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
  6/782 [..............................] - ETA: 1:17:22 - loss: 4.4283 - accuracy: 0.4886

KeyboardInterrupt: ignored

In [62]:
# plot the loss
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

NameError: ignored

In [22]:
# save it as a h5 file


from tensorflow.keras.models import load_model

model.save('model_Restnet50.h5')

In [23]:
model.save('/content/drive/MyDrive/Datasets/model_Restnet50.h5')

In [67]:

y_pred = model.predict(test_set)


In [25]:
y_pred

array([[0.00294292, 0.9970571 ],
       [0.01994183, 0.9800582 ],
       [0.00294629, 0.99705374],
       ...,
       [0.0036795 , 0.9963205 ],
       [0.00302054, 0.9969795 ],
       [0.00946037, 0.99053967]], dtype=float32)

In [26]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [27]:
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [69]:
print(y_pred)

[[0.00795667 0.9920433 ]
 [0.01415128 0.98584867]
 [0.02194767 0.9780523 ]
 ...
 [0.00865988 0.9913401 ]
 [0.00667278 0.99332726]
 [0.0133619  0.98663807]]


In [6]:
from tensorflow.keras.models import load_model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [7]:
#model=load_model('model_resnet50.h5')
model=load_model('model_Restnet50.h5')

OSError: ignored

In [8]:

model=load_model('/content/drive/MyDrive/Datasets/model_Restnet50.h5')

In [52]:
img=image.load_img('/content/drive/MyDrive/Datasets/test/cat/100.jpg',target_size=(224,224))



In [53]:
x=image.img_to_array(img)
x

array([[[  4.,   4.,   4.],
        [  2.,   2.,   2.],
        [  2.,   2.,   2.],
        ...,
        [ 11.,  11.,  11.],
        [ 10.,  10.,  10.],
        [ 19.,  19.,  19.]],

       [[  4.,   4.,   4.],
        [  2.,   2.,   2.],
        [  2.,   2.,   2.],
        ...,
        [ 11.,  11.,  11.],
        [ 10.,  10.,  10.],
        [ 22.,  22.,  22.]],

       [[  4.,   4.,   4.],
        [  2.,   2.,   2.],
        [  2.,   2.,   2.],
        ...,
        [ 11.,  11.,  11.],
        [ 10.,  10.,  10.],
        [ 22.,  22.,  22.]],

       ...,

       [[145., 185., 193.],
        [149., 188., 195.],
        [160., 197., 203.],
        ...,
        [248., 255., 255.],
        [249., 255., 255.],
        [253., 253., 253.]],

       [[153., 193., 201.],
        [153., 192., 199.],
        [160., 197., 203.],
        ...,
        [248., 255., 255.],
        [249., 255., 255.],
        [254., 254., 254.]],

       [[150., 190., 198.],
        [159., 198., 205.],
        [178., 2

In [54]:
x.shape

(224, 224, 3)

In [55]:
x=x/255

In [56]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [57]:
model.predict(img_data)

array([[0.29251972, 0.7074803 ]], dtype=float32)

In [58]:
a=np.argmax(model.predict(img_data), axis=1)

In [59]:
print(a)

[1]


In [60]:
a==1

array([ True])

In [61]:
a==0

array([False])